In [7]:
import numpy as np 
import cv2 as cv

target_image = cv.imread("image1.jpg",cv.COLOR_BGR2RGB)
projecting_image = cv.imread("img3.png",cv.COLOR_BGR2RGB)

target_image = cv.resize(target_image,(200,300))
projecting_image = cv.resize(projecting_image,(200,300))



# converting to grayscale

gray_target = cv.cvtColor(target_image,cv.COLOR_BGR2GRAY)



orb = cv.ORB.create(1000)

# Brute-Force Matching with ORB
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

vid = cv.VideoCapture("video1.mp4")     #sizeiisue

while True:
    yesNo,frame = vid.read()
    # frame = cv.resize(frame,(600,450))
    if yesNo == False:
        break

    gray_scan = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    kp1,des1 = orb.detectAndCompute(gray_target,None)
    kp2,des2 = orb.detectAndCompute(gray_scan,None)

    matches = bf.knnMatch(des1,des2,k=1)
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    matches = bf.match(des1,des2)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    if len(matches)>100:

        # to convert Keypoints (from a list of matches) to an argument for findHomography().
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
        m, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)

        h,w,z = frame.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                
        img_coordinate = np.float32([[0,0],[0,299],[150,50],[299,0]]).reshape(-1,1,2)

        live_coordinate = cv.perspectiveTransform(img_coordinate,m)

        augmented_frame = cv.warpPerspective(projecting_image,m,(600,450))

        f_area = cv.fillConvexPoly(frame,dst_pts.astype(int),(0,0,0))
        f_area = cv.resize(f_area,(600,450))

        final_frame = f_area+augmented_frame
        cv.imshow("img",final_frame)



    if cv.waitKey(1) == ord("q"):
        break

vid.release()
cv.destroyAllWindows()

In [7]:
target_image.shape

(300, 200, 3)